## Preambule

In [11]:
# General packages
import numpy as np
import pandas as pd
from tqdm import tqdm
from pathlib import Path
import xarray as xr
import json

# Plotting
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from plotly.colors import n_colors

## Read

In [12]:
path_main = Path("K:/Code/EffortSharing/")
path_data = Path("K:/Data/Data_EffortSharing/DataUpdate_ongoing/")
path_figs = path_main / "Figures" / "NZreport"

In [13]:
xr_dataread = xr.open_dataset(path_data / "xr_dataread.nc")
all_regions_iso = np.load(path_data / "all_regions.npy")
all_regions_names = np.load(path_data / "all_regions_names.npy")
all_countries_iso = np.load(path_data / "all_countries.npy", allow_pickle=True)
all_countries_names = np.load(path_data / "all_countries_names.npy", allow_pickle=True)

In [36]:
settings_default =  {'Temperature': 1.6,
                   'Risk': [0.5, 0.33, 0.17],
                   'NegEmis': 0.5,
                   'NonCO2red': [0.33, 0.5, 0.67],
                   'Timing': "Immediate", 
                   'Convergence_year': [2040, 2050, 2060, 2070],}
settings_default_global = {'Temperature': 1.6,
                   'Risk': [0.5, 0.33, 0.17],
                   'NegEmis': 0.5,
                   'NonCO2red': [0.33, 0.5, 0.67],
                   'Timing': "Immediate"}

In [297]:
major_countries = ['USA', 'EU', 'CHN', 'IND', 'BRA', 'RUS', 'JPN', 'AUS', 'ZAF']

In [298]:
xr_allocs = []
for cty in major_countries:#list(all_countries_iso)+['EU']+['EARTH']:
    xr_allocs.append(xr.open_dataset(path_data / "Allocations_CO2_excl" / ("xr_alloc_"+cty+".nc")).expand_dims({'Region':[cty]}).sel(**settings_default)[['GF', 'ECPC', 'PCC', 'AP', 'PC']])
xr_allocs = xr.merge(xr_allocs)

# xr_allocs_incl = []
# for cty in ['EU', 'USA', 'IND', 'CHN', 'JPN', 'AUS', 'BRA', 'RUS', 'ZAF']:
#     xr_allocs_incl.append(xr.open_dataset(path_data / "Allocations_CO2_incl" / ("xr_alloc_"+cty+".nc")).expand_dims({'Region':[cty]})[['GF', 'ECPC', 'PCC', 'AP', 'PC']].sel(Temperature=1.6, Risk=0.5, NegEmis=0.5, NonCO2red=0.5, Timing='Immediate', Scenario='SSP2'))
# xr_allocs_incl = xr.merge(xr_allocs_incl)

In [299]:
reds_2040 = -(xr_allocs.sel(Time=2040) - xr_dataread.sel(Time=2015).CO2_hist_excl) / xr_dataread.sel(Time=2015).CO2_hist_excl
reds_2030 = -(xr_allocs.sel(Time=2030) - xr_dataread.sel(Time=2015).CO2_hist_excl) / xr_dataread.sel(Time=2015).CO2_hist_excl

In [300]:
policy_budget = 25*0.5*xr_dataread.CO2_hist_excl.sel(Time=2015)

In [301]:
xr_bud = xr_allocs.where(xr_allocs >=0, 0).sum(dim='Time')
debtpc = (xr_bud - policy_budget) / xr_dataread.Population.sel(Time=2021, Scenario='SSP2')

In [302]:
vals_2030_pcc = []
vals_2030_ecpc = []
vals_2030_ap = []
vals_2040_pcc = []
vals_2040_ecpc = []
vals_2040_ap = []
regs_pcc = []
regs_ecpc = []
regs_ap = []
debt_pcc = []
debt_ecpc = []
debt_ap = []
for cty in major_countries:
    vals_2030_pcc = vals_2030_pcc + list(reds_2030.sel(Region=cty).PCC.values.flatten())
    vals_2030_ecpc = vals_2030_ecpc + list(reds_2030.sel(Region=cty).ECPC.values.flatten())
    vals_2030_ap = vals_2030_ap + list(reds_2030.sel(Region=cty).AP.values.flatten())

    vals_2040_pcc = vals_2040_pcc + list(reds_2040.sel(Region=cty).PCC.values.flatten())
    vals_2040_ecpc = vals_2040_ecpc + list(reds_2040.sel(Region=cty).ECPC.values.flatten())
    vals_2040_ap = vals_2040_ap + list(reds_2040.sel(Region=cty).AP.values.flatten())

    debt_pcc = debt_pcc + list(debtpc.sel(Region=cty).PCC.values.flatten())
    debt_ecpc = debt_ecpc + list(debtpc.sel(Region=cty).ECPC.values.flatten())
    debt_ap = debt_ap + list(debtpc.sel(Region=cty).AP.values.flatten())

    regs_pcc = regs_pcc + [all_regions_names[all_regions_iso[:-8] == cty][0]]*len(reds_2030.sel(Region=cty).PCC.values.flatten())
    regs_ecpc = regs_ecpc + [all_regions_names[all_regions_iso[:-8] == cty][0]]*len(reds_2030.sel(Region=cty).ECPC.values.flatten())
    regs_ap = regs_ap + [all_regions_names[all_regions_iso[:-8] == cty][0]]*len(reds_2030.sel(Region=cty).AP.values.flatten())

    #vals_2040 = vals_2040 + list(reds_2030.PCC.values.flatten())+list(reds_2030.ECPC.values.flatten())+list(reds_2030.AP.values.flatten())
    #regs = regs + [cty]*(len(reds_2030.PCC.values.flatten())+len(reds_2030.ECPC.values.flatten())+len(reds_2030.AP.values.flatten()))

## Plot

In [307]:
fig = make_subplots(rows=1, cols=3,
                    specs=[[{}, {}, {}]],
                    subplot_titles=("<b>CO<sub>2</sub> reduction targets<br>by 2030", "<b>CO<sub>2</sub> reduction targets<br>by 2040", "<b>Cumulative CO<sub>2</sub><br>emissions"),
                    horizontal_spacing = 0.05, vertical_spacing=0.05, shared_xaxes=False, shared_yaxes=True)

fig.add_trace(go.Box(x=vals_2030_pcc, y=regs_pcc, name = 'Equality', boxpoints=False, offsetgroup='A', marker_color='rgba(61, 153, 112, 1)', line_width=1), 1, 1)
fig.add_trace(go.Box(x=vals_2030_ap, y=regs_ap,name = 'Capability',boxpoints=False, offsetgroup='B', marker_color='rgba(255, 65, 54, 1)', line_width=1), 1, 1)
fig.add_trace(go.Box(x=vals_2030_ecpc, y=regs_ecpc, name = 'Responsibility',boxpoints=False, offsetgroup='C', marker_color='rgba(255, 133, 27,1)', line_width=1), 1, 1)
fig.add_vline(x=0, line_width=2, line_dash="dash", line_color="black")
fig.add_vline(x=1, line_width=2, line_dash="dash", line_color="black")

fig.add_trace(go.Box(x=vals_2040_pcc, y=regs_pcc, offsetgroup='A', showlegend=False, boxpoints=False, marker_color='rgba(61, 153, 112, 1)', line_width=1), 1, 2)
fig.add_trace(go.Box(x=vals_2040_ap, y=regs_ap, offsetgroup='B', showlegend=False, boxpoints=False, marker_color='rgba(255, 65, 54, 1)', line_width=1), 1, 2)
fig.add_trace(go.Box(x=vals_2040_ecpc, y=regs_ecpc, offsetgroup='C', showlegend=False, boxpoints=False,  marker_color='rgba(255, 133, 27,1)', line_width=1), 1, 2)
fig.add_vline(x=0, line_width=2, line_dash="dash", line_color="black")
fig.add_vline(x=1, line_width=2, line_dash="dash", line_color="black")

fig.add_trace(go.Box(x=debt_pcc, y=regs_pcc, offsetgroup='A', showlegend=False, boxpoints=False, marker_color='rgba(61, 153, 112, 1)', line_width=1), 1, 3)
fig.add_trace(go.Box(x=debt_ap, y=regs_ap, offsetgroup='B', showlegend=False, boxpoints=False, marker_color='rgba(255, 65, 54, 1)', line_width=1), 1, 3)
fig.add_trace(go.Box(x=debt_ecpc, y=regs_ecpc, offsetgroup='C', showlegend=False, boxpoints=False,  marker_color='rgba(255, 133, 27,1)', line_width=1), 1, 3)
fig.add_vline(x=0, line_width=2, line_dash="dash", line_color="black")

fig.update_layout(
    boxmode='group', # group together boxes of the different traces for each value of x
      boxgap=0.4,       # Space between different categories (days)
    boxgroupgap=0.1,
        yaxis=dict(tickfont=dict(size=16)),   # Increase tick font size for first subplot
        yaxis2=dict(tickfont=dict(size=16), showline=False, ticks='')
)

for i in range(3):
  fig.update_xaxes(
      tickvals=[[0, 0.2, 0.4, 0.6, 0.8, 1.0, 1.2], [0, 0.2, 0.4, 0.6, 0.8, 1.0, 1.2], [-100, 0, 100]][i],  # Custom tick positions
      ticktext=[['0%', '20%', '40%', '60%', '80%', 'Net<br>zero', '120%'], ['0%', '20%', '40%', '60%', '80%', 'Net<br>zero', '120%'], ['150 t CO<sub>2</sub>/cap<br>debt', 'No debt<br>nor credit', '150 t CO<sub>2</sub>/cap<br>credit']][i],  # Custom tick labels
      row=1, col=1+i,
      range=[[-0.2, 1.5], [-0.2, 1.5], [-150, 150]][i],
      title_text = ["Reduction relative to 2015", "Reduction relative to 2015", "Carbon debt (left) or credit (right) per capita"][i],
  )

fig.update_traces(orientation='h')
for annotation in fig['layout']['annotations']:
    annotation['font'] = dict(size=25)
fig.update_layout(height=1300, width=1600, template='simple_white',
                  legend=dict(
                     x=-0.15, y=1.08,
        font=dict(size=20)
    ))
fig.write_image(path_figs / "Figure_2.png", scale=4)
fig.show()